# LAB 3b:  BigQuery ML Model Linear Feature Engineering/Transform.

**Learning Objectives**

1. Create and evaluate linear model with BigQuery's ML.FEATURE_CROSS
1. Create and evaluate linear model with BigQuery's ML.FEATURE_CROSS and ML.BUCKETIZE
1. Create and evaluate linear model with ML.TRANSFORM


## Introduction 
In this notebook, we will create multiple linear models to predict the weight of a baby before it is born, using increasing levels of feature engineering using BigQuery ML. We saw this two courses ago applying advanced feature engineering with BQML but instead with the [taxifare dataset](../../feature_engineering/solutions/bqml_adv_feat_eng.ipynb).

In this lab, we will create and evaluate a linear model using BigQuery's ML.FEATURE_CROSS, create and evaluate a linear model using BigQuery's ML.FEATURE_CROSS and ML.BUCKETIZE, and create and evaluate a linear model using BigQuery's ML.TRANSFORM.

Each learning objective will correspond to a __#TODO__ in this student lab notebook -- try to complete this notebook first and then review the [solution notebook](../solutions/5_bqml_linear_transform_babyweight.ipynb).

## Load necessary libraries

Check that the Google BigQuery library is installed and if not, install it. 

In [1]:
%%bash
pip freeze | grep google-cloud-bigquery==1.6.1 || \
pip install google-cloud-bigquery==1.6.1

google-cloud-bigquery==1.6.1


## Verify tables exist

Verify that you previously created the dataset and data tables. If not, go back to lab [2_prepare_babyweight](../solutions/2_prepare_babyweight.ipynb) to create them.


In [2]:
%%bigquery
-- LIMIT 0 is a free query; this allows us to check that the table exists.
SELECT * FROM babyweight.babyweight_data_train
LIMIT 0

,weight_pounds,is_male,mother_age,plurality,gestation_weeks


In [3]:
%%bigquery
-- LIMIT 0 is a free query; this allows us to check that the table exists.
SELECT * FROM babyweight.babyweight_data_eval
LIMIT 0

,weight_pounds,is_male,mother_age,plurality,gestation_weeks


## Model 1:  Apply the ML.FEATURE_CROSS clause to categorical features

BigQuery ML now has ML.FEATURE_CROSS, a pre-processing clause that performs a feature cross.  

* ML.FEATURE_CROSS generates a STRUCT feature with all combinations of crossed categorical features, except for 1-degree items (the original features) and self-crossing items.  

* Syntax:  ML.FEATURE_CROSS(STRUCT(features), degree)

* The feature parameter is a categorical features separated by comma to be crossed. The maximum number of input features is 10. An unnamed feature is not allowed in features. Duplicates are not allowed in features.

* Degree(optional): The highest degree of all combinations. Degree should be in the range of [1, 4]. Default to 2.

Output: The function outputs a STRUCT of all combinations except for 1-degree items (the original features) and self-crossing items, with field names as concatenation of original feature names and values as the concatenation of the column string values.

#### INSTRUCTOR NOTE ONLY:  
Why ML.Feature_Cross uses STRUCT - All elements in an array have the same size because all elements are of the same datatype whereas STRUCT can contain elements of dissimilar datatype.  Hence, all elements are of different size.  For example, is_male and plurality are dissimilar datatypes.

#### Create model with feature cross.

In [4]:
%%bigquery
CREATE OR REPLACE MODEL
    babyweight.model_1

OPTIONS (
    MODEL_TYPE="LINEAR_REG",
    INPUT_LABEL_COLS=["weight_pounds"],
    L2_REG=0.1,
    DATA_SPLIT_METHOD="NO_SPLIT") AS

SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks,
    ML.FEATURE_CROSS(
        STRUCT(
            is_male,
            plurality)
    ) AS gender_plurality_cross
FROM
    babyweight.babyweight_data_train

""


#### Create two SQL statements to evaluate the model.

In [5]:
%%bigquery
SELECT
    *
FROM
    ML.EVALUATE(MODEL babyweight.model_1,
    (
    SELECT
        weight_pounds,
        is_male,
        mother_age,
        plurality,
        gestation_weeks,
        ML.FEATURE_CROSS(
            STRUCT(
                is_male,
                plurality)
        ) AS gender_plurality_cross
    FROM
        babyweight.babyweight_data_eval
    ))

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,0.829803,1.139565,0.02041,0.67478,0.345091,0.345093


In [6]:
%%bigquery
SELECT
    SQRT(mean_squared_error) AS rmse
FROM
    ML.EVALUATE(MODEL babyweight.model_1,
    (
    SELECT
        weight_pounds,
        is_male,
        mother_age,
        plurality,
        gestation_weeks,
        ML.FEATURE_CROSS(
            STRUCT(
                is_male,
                plurality)
        ) AS gender_plurality_cross
    FROM
        babyweight.babyweight_data_eval
    ))

,rmse
0,1.067504


## BQML's Pre-processing functions:

Here are some of the preprocessing functions in BigQuery ML:
* ML.FEATURE_CROSS(STRUCT(features))    does a feature cross of all the combinations
* ML.POLYNOMIAL_EXPAND(STRUCT(features), degree)    creates x, x^2, x^3, etc.
* ML.BUCKETIZE(f, split_points)   where split_points is an array 

## Model 2:  Apply the BUCKETIZE Function 


##### BUCKETIZE 
Bucketize is a pre-processing function that creates "buckets" (e.g bins) - e.g. it bucketizes a continuous numerical feature into a string feature with bucket names as the value.

* ML.BUCKETIZE(feature, split_points)

* feature: A numerical column.

* split_points: Array of numerical points to split the continuous values in feature into buckets. With n split points (s1, s2 … sn), there will be n+1 buckets generated. 

* Output: The function outputs a STRING for each row, which is the bucket name. bucket_name is in the format of bin_<bucket_number>, where bucket_number starts from 1.

#### Apply the BUCKETIZE function within FEATURE_CROSS.
* Hint:  Create a model_2.

In [7]:
%%bigquery
CREATE OR REPLACE MODEL
    babyweight.model_2

OPTIONS (
    MODEL_TYPE="LINEAR_REG",
    INPUT_LABEL_COLS=["weight_pounds"],
    L2_REG=0.1,
    DATA_SPLIT_METHOD="NO_SPLIT") AS

SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks,
    ML.FEATURE_CROSS(
        STRUCT(
            is_male,
            ML.BUCKETIZE(
                mother_age,
                GENERATE_ARRAY(15, 45, 1)
            ) AS bucketed_mothers_age,
            plurality,
            ML.BUCKETIZE(
                gestation_weeks,
                GENERATE_ARRAY(17, 47, 1)
            ) AS bucketed_gestation_weeks
        )
    ) AS crossed
FROM
    babyweight.babyweight_data_train

""


#### Create three SQL statements to EVALUATE the model.

In [8]:
%%bigquery
SELECT * FROM ML.TRAINING_INFO(MODEL babyweight.model_2)

,training_run,iteration,loss,eval_loss,learning_rate,duration_ms
0,0,0,1.056824,None,None,38752


In [9]:
%%bigquery
SELECT
    *
FROM
    ML.EVALUATE(MODEL babyweight.model_2,
    (
    SELECT
        weight_pounds,
        is_male,
        mother_age,
        plurality,
        gestation_weeks,
        ML.FEATURE_CROSS(
            STRUCT(
                is_male,
                ML.BUCKETIZE(
                    mother_age,
                    GENERATE_ARRAY(15, 45, 1)
                ) AS bucketed_mothers_age,
                plurality,
                ML.BUCKETIZE(
                    gestation_weeks,
                    GENERATE_ARRAY(17, 47, 1)
                ) AS bucketed_gestation_weeks
            )
        ) AS crossed
    FROM
        babyweight.babyweight_data_eval))

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,0.798202,1.057588,0.017775,0.650903,0.392203,0.392205


In [10]:
%%bigquery
SELECT
    SQRT(mean_squared_error) AS rmse
FROM
    ML.EVALUATE(MODEL babyweight.model_2,
    (
    SELECT
        weight_pounds,
        is_male,
        mother_age,
        plurality,
        gestation_weeks,
        ML.FEATURE_CROSS(
            STRUCT(
                is_male,
                ML.BUCKETIZE(
                    mother_age,
                    GENERATE_ARRAY(15, 45, 1)
                ) AS bucketed_mothers_age,
                plurality,
                ML.BUCKETIZE(
                    gestation_weeks,
                    GENERATE_ARRAY(17, 47, 1)
                ) AS bucketed_gestation_weeks
            )
        ) AS crossed
    FROM
        babyweight.babyweight_data_eval))

,rmse
0,1.028391


## Model 3:  Apply the TRANSFORM clause

Before we perform our prediction, we should encapsulate the entire feature set in a TRANSFORM clause.  BigQuery ML now supports defining data transformations during model creation, which will be automatically applied during prediction and evaluation. This is done through the TRANSFORM clause in the existing CREATE MODEL statement. By using the TRANSFORM clause, user specified transforms during training will be automatically applied during model serving (prediction, evaluation, etc.) 

In our case, we are using the TRANSFORM clause to separate out the raw input data from the TRANSFORMED features.  The input columns of the TRANSFORM clause is the query_expr (AS SELECT part).  The output columns of TRANSFORM from select_list are used in training. These transformed columns are post-processed with standardization for numerics and one-hot encoding for categorical variables by default. 

The advantage of encapsulating features in the TRANSFORM clause is the client code doing the PREDICT doesn't change, e.g. our model improvement is transparent to client code. Note that the TRANSFORM clause MUST be placed after the CREATE statement.

#### Apply the TRANSFORM clause to the model_3 and run the query.

In [11]:
%%bigquery
CREATE OR REPLACE MODEL
    babyweight.model_3

TRANSFORM(
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks,
    ML.FEATURE_CROSS(
        STRUCT(
            is_male,
            ML.BUCKETIZE(
                mother_age,
                GENERATE_ARRAY(15, 45, 1)
            ) AS bucketed_mothers_age,
            plurality,
            ML.BUCKETIZE(
                gestation_weeks,
                GENERATE_ARRAY(17, 47, 1)
            ) AS bucketed_gestation_weeks
        )
    ) AS crossed
)

OPTIONS (
    MODEL_TYPE="LINEAR_REG",
    INPUT_LABEL_COLS=["weight_pounds"],
    L2_REG=0.1,
    DATA_SPLIT_METHOD="NO_SPLIT") AS

SELECT
    *
FROM
    babyweight.babyweight_data_train

""


#### Create three SQL statements to EVALUATE model_3.

In [12]:
%%bigquery
SELECT * FROM ML.TRAINING_INFO(MODEL babyweight.model_3)

,training_run,iteration,loss,eval_loss,learning_rate,duration_ms
0,0,0,1.056824,None,None,40302


In [13]:
%%bigquery
SELECT
    *
FROM
    ML.EVALUATE(MODEL babyweight.model_3,
    (
    SELECT
        *
    FROM
        babyweight.babyweight_data_eval
    ))

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,0.798202,1.057588,0.017775,0.650902,0.392203,0.392205


In [14]:
%%bigquery
SELECT
    SQRT(mean_squared_error) AS rmse
FROM
    ML.EVALUATE(MODEL babyweight.model_3,
    (
    SELECT
        *
    FROM
        babyweight.babyweight_data_eval
    ))

,rmse
0,1.028391


## Lab Summary: 
In this lab, we created and evaluated a linear model using BigQuery's ML.FEATURE_CROSS, created and evaluated a linear model using BigQuery's ML.FEATURE_CROSS and ML.BUCKETIZE, and created and evaluated a linear model using BigQuery's ML.TRANSFORM.

Copyright 2019 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License